In [61]:
import cobra_postgres
import cobra_datamodelfabric
import uuid
import time
import cobra_jobregistry
import cobra_gdal
import json
import requests
import os
import pandas as pd

In [ ]:
reg = cobra_jobregistry.cobra_jobregistry()

In [ ]:
files = ['gis_osm_buildings_a_free_1.shp',
'gis_osm_landuse_a_free_1.shp',
'gis_osm_natural_a_free_1.shp',
'gis_osm_natural_free_1.shp',
'gis_osm_places_a_free_1.shp',
'gis_osm_places_free_1.shp',
'gis_osm_pofw_a_free_1.shp',
'gis_osm_pofw_free_1.shp',
'gis_osm_pois_a_free_1.shp',
'gis_osm_pois_free_1.shp',
'gis_osm_railways_free_1.shp',
'gis_osm_roads_free_1.shp',
'gis_osm_traffic_a_free_1.shp',
'gis_osm_traffic_free_1.shp',
'gis_osm_transport_a_free_1.shp',
'gis_osm_transport_free_1.shp',
'gis_osm_water_a_free_1.shp',
'gis_osm_waterways_free_1.shp']

In [ ]:
for a_file in files:
    reg.create_shape2pg_job(f'/download/{a_file}')

In [ ]:
for a_file in files:
    body = {'path':f'/download/{a_file}'}
    r = requests.post('http://jobregistry:5000/job', json=body)
    r.text

In [ ]:
gdal = cobra_gdal.cobra_gdal(schema='osm', run_in_loop=True)

In [43]:
import requests

In [44]:
r = requests.get('http://jobregistry:5000/')
r.text

'Hello, Docker!'

In [ ]:
body = {'path':'/highway/to/homber','job_name':'HOmberg', 'skip_failures':'False','priority':27}
r = requests.post('http://jobregistry:5000/job', json=body)
r.text

In [60]:
class Job():
    
    def __init__(self):
        
        self.id : UUID = None
        self.name : String = None
        self.job_type : String = None
        self.date_created : Date = None
        self.date_stated : Date = None
        self.date_finished : Date = None
        self.priority : Integer = None
        self.status : String = None
    
    def set_values_from_db(self, job_id, name, job_type, date_created, date_started, date_finished, priority, status):
        
        self.id : UUID = job_id
        self.name : String = name
        self.job_type : String = job_type
        self.date_created : Date = date_created
        self.date_stated : Date = date_started
        self.date_finished : Date = date_finished
        self.priority : Integer = priority
        self.status : String = status
        
        
    def values_as_list(self):
             
        return [self.id,
        self.name,
        self.job_type,
        self.date_created,
        self.date_stated,
        self.date_finished,
        self.priority,
        self.status]


In [66]:
class Job_Manager():
    
    def __init__(self):
        
        self.pg_interface = cobra_postgres.PgInterface()
        self.pg_interface.switch_schema('gdal')
    
    def get_jobs(self, df=False):
        
        job_list = []
        with self.pg_interface.get_connection() as conn:
            
            with conn.cursor() as curs:
                
                curs.execute('SELECT * FROM gdal.gdal_jobs')
                results = curs.fetchall()
                for a_result in results:
                    new_job = Job()
                    new_job.set_values_from_db(a_result[0], a_result[1],a_result[2],a_result[3],a_result[4],a_result[5],a_result[6],a_result[7])
                    job_list.append(new_job)
                
        if df == False:
        
            return job_list
        
        else:
            
            return pd.DataFrame([a_job.values_as_list() for a_job in job_list])
    
    def create_new_shape_to_pg(self, path, job_name = None, skip_failures = None, priority = None):
        
        # Check if file exists
        if os.path.isfile(path) == False:
            
            raise Exception(f'File {path} does not exist')
        
        body = {'path' : path}
        if job_name != None:
            body['job_name'] = job_name
        if skip_failures != None:
            body['skip_failures'] = skip_failures
        if priority != None:
            body['priority'] = priority
    
        r = requests.post('http://jobregistry:5000/job', json=body)
        
        if r.status_code != 200:
            raise Exception(f'job server has not accepted the job')


In [67]:
job_manager = Job_Manager()

In [54]:
job_manager.create_new_shape_to_pg('/download/gis_osm_buildings_a_free_1.shp')

{
  "path": "/download/gis_osm_buildings_a_free_1.shp"
}

200


In [68]:
job_manager.get_jobs(df=True)

,0,1,2,3,4,5,6,7
0,7e89912e-1b1e-4117-bce9-7e3d632e83b3,Load Shape,shape2pg,2021-12-19 12:39:42.687793,NaT,NaT,42,Failed
1,26f41c60-1e19-4438-a827-d249d7ac5495,Load Shape,shape2pg,2021-12-19 12:39:42.793483,NaT,NaT,42,Failed
2,bb3c5f6f-f17a-4336-b370-24da55ab6b1a,Load Shape,shape2pg,2021-12-19 12:39:42.898328,NaT,NaT,42,Failed
3,6d1c5347-ddc9-46b8-9881-4fc5725950b2,Load Shape,shape2pg,2021-12-19 12:39:43.001517,NaT,NaT,42,Failed
4,ff04ff52-3459-4853-937d-38a2e180968e,Load Shape,shape2pg,2021-12-19 12:39:43.094485,NaT,NaT,42,Failed
5,8de006f2-a073-4286-99c9-f3fc8704c931,Load Shape,shape2pg,2021-12-19 12:39:43.185158,NaT,NaT,42,Failed
6,d889e21b-c7b8-4792-9e18-0fdf7a562d6e,Load Shape,shape2pg,2021-12-19 12:39:43.280975,NaT,NaT,42,Failed
7,be437275-fe8f-4502-bdbd-fc52bb66f472,Load Shape,shape2pg,2021-12-19 12:39:43.382729,NaT,NaT,42,Failed
8,f48e8a72-d4ac-42e1-9708-dfa31a7c0bf8,Load Shape,shape2pg,2021-12-19 12:39:43.488058,NaT,NaT,42,Failed
9,ab8eb24b-2011-46e9-88ae-b41ccd92c582,Load Shape,shape2pg,2021-12-19 12:39:43.581072,NaT,NaT,42,Failed


In [53]:
! ls /download

gis_osm_buildings_a_free_1.cpg	gis_osm_pois_free_1.dbf
gis_osm_buildings_a_free_1.dbf	gis_osm_pois_free_1.prj
gis_osm_buildings_a_free_1.prj	gis_osm_pois_free_1.shp
gis_osm_buildings_a_free_1.shp	gis_osm_pois_free_1.shx
gis_osm_buildings_a_free_1.shx	gis_osm_railways_free_1.cpg
gis_osm_landuse_a_free_1.cpg	gis_osm_railways_free_1.dbf
gis_osm_landuse_a_free_1.dbf	gis_osm_railways_free_1.prj
gis_osm_landuse_a_free_1.prj	gis_osm_railways_free_1.shp
gis_osm_landuse_a_free_1.shp	gis_osm_railways_free_1.shx
gis_osm_landuse_a_free_1.shx	gis_osm_roads_free_1.cpg
gis_osm_natural_a_free_1.cpg	gis_osm_roads_free_1.dbf
gis_osm_natural_a_free_1.dbf	gis_osm_roads_free_1.prj
gis_osm_natural_a_free_1.prj	gis_osm_roads_free_1.shp
gis_osm_natural_a_free_1.shp	gis_osm_roads_free_1.shx
gis_osm_natural_a_free_1.shx	gis_osm_traffic_a_free_1.cpg
gis_osm_natural_free_1.cpg	gis_osm_traffic_a_free_1.dbf
gis_osm_natural_free_1.dbf	gis_osm_traffic_a_free_1.prj
gis_osm_natural_free_1.prj	gis_osm_traffic_a_free_1.s